# Association Rules

The input for our data transformation will be the cleaned dataset generated by running `cleaner.py`

In [1]:
import numpy as np
import pandas as pd

df = pd.read_csv("../../data_transformation/nft_transactions_cleaned.csv", parse_dates=["full_date"])

df

C:\Users\Alex\AppData\Local\Temp\ipykernel_11512\3423358185.py:4: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../../data_transformation/nft_transactions_cleaned.csv", parse_dates=["full_date"])


,full_date,transaction_id,asset_id,asset_name,collection_name,wei_price,payment_token_name,quantity,seller_address,seller_username,buyer_address,asset_category,day_name,day_of_week,month_name,month_number,year,time,hour
0,2019-01-01,7453402,0,Unknown,Unknown,150000000000000000,Ether,0,0xf8856124ea157d6f26472e38224ee6744ad13af3,Farm4Ace,0x7ebc7e83cb2b43deafa82eb0ae8b5daea3fe9a13,Uncategorized,Tuesday,1,January,1,2019,15:48:57,15
1,2019-01-01,7461241,0,Unknown,Unknown,120000000000000000,Ether,0,0xf29db2342a9dcb1bba4f7f2b1029bdd8ae5c0668,Unknown,0x8691271d453022a447320ed82bcb2a30ba949d03,Uncategorized,Tuesday,1,January,1,2019,23:50:55,23
2,2019-01-02,7471818,0,Unknown,Unknown,100000000000000000,Ether,0,0x057dc43af6187a931e78a56b3293912a5b42e5ec,hagoromo,0xff60e23eb04b76c699e3d180617c3245cbcafd78,Uncategorized,Wednesday,2,January,1,2019,16:44:58,16
3,2019-01-03,7485269,0,Unknown,Unknown,10000000000000000,Ether,0,0x112ab3e7440b6e6b5c0fe0245579755dd78285b3,Unknown,0xa58d0065cb381c80db1487e09ec803cd26a26264,Uncategorized,Thursday,3,January,1,2019,06:31:46,6
4,2019-01-03,7487454,0,Unknown,Unknown,249580457577544250,Ether,0,0x889463ce0946ec870c6e9a5a42a0e85c6d1a20fd,OffTheBlockChainAxies,0x445ba6f9f553872fa9cdc14f5c0639365b39c140,Uncategorized,Thursday,3,January,1,2019,10:31:43,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5252249,2021-11-29,2232084052,115052697,"#86 - ""Fret For Your Prozac""",~MyriadMeaning~,0,Ether,1,0x14b78b9fe0af98145e57f2a1b8a66044ab6a9eff,PiratesBootyGallery,0xaeb2707eda1e201cb13aa80b915bc3979d7e33a0,Uncategorized,Monday,0,November,11,2021,07:27:49,7
5252250,2021-12-03,2301205938,102476305,#67 - MOLD,~MyriadMeaning~,8000000000000000,Ether,1,0x609b9fc66519fb5fa855b8427fb482bca63cd423,KillMePlease,0x42b018ffce9498915e530ca0f871d6f7144b9acc,Uncategorized,Friday,4,December,12,2021,07:06:44,7
5252251,2021-03-06,82596530,18310514,A Gentle Breeze,~~ A e s t h e t i c ~~,20000000000000000,Wrapped Ether,1,0x974a344968786201a5f2e282014098f1333aa73b,VisualSensation,0xe7e30b515304b0d4e2cf919c449f1d2ec77b1319,Uncategorized,Saturday,5,March,3,2021,06:09:07,6
5252252,2021-03-29,94187279,19807908,Gas Fees,~~ A e s t h e t i c ~~,10000000000000000,Ether,1,0x974a344968786201a5f2e282014098f1333aa73b,VisualSensation,0x263108604e0c9e1f6cdcf698e160cd563635a328,Uncategorized,Monday,0,March,3,2021,11:33:23,11


To create the input for the association rules we only have to keep the following columns: `buyer_address` and `asset_category`

In [2]:
association = df[['buyer_address', 'asset_category']]
association

,buyer_address,asset_category
0,0x7ebc7e83cb2b43deafa82eb0ae8b5daea3fe9a13,Uncategorized
1,0x8691271d453022a447320ed82bcb2a30ba949d03,Uncategorized
2,0xff60e23eb04b76c699e3d180617c3245cbcafd78,Uncategorized
3,0xa58d0065cb381c80db1487e09ec803cd26a26264,Uncategorized
4,0x445ba6f9f553872fa9cdc14f5c0639365b39c140,Uncategorized
...,...,...
5252249,0xaeb2707eda1e201cb13aa80b915bc3979d7e33a0,Uncategorized
5252250,0x42b018ffce9498915e530ca0f871d6f7144b9acc,Uncategorized
5252251,0xe7e30b515304b0d4e2cf919c449f1d2ec77b1319,Uncategorized
5252252,0x263108604e0c9e1f6cdcf698e160cd563635a328,Uncategorized


In [3]:
# Create a column to count purchases. I could use the original quantity column but it doesn't matter.
# The input for the association rules will always be true/false.
association['purchase'] = 1

# Pivot the table
pivot_table = association.pivot_table(index='buyer_address', 
                             columns='asset_category', 
                             values='purchase', 
                             aggfunc='sum', 
                             fill_value=0)

# Reset the index 
pivot_table = pivot_table.reset_index()
# Remove the index name
pivot_table = pivot_table.rename_axis(None, axis=1)
pivot_table

C:\Users\Alex\AppData\Local\Temp\ipykernel_11512\1780440310.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  association['purchase'] = 1


,buyer_address,Art,Collectibles,Domain,Music,Photography,Sports,Trading Cards,Uncategorized,Utility,Virtual Worlds
0,0x00000000000040c8d72ad3a15ce5408f99cd61b4,0,0,1,0,0,0,0,0,0,0
1,0x0000000000004c0fd5233e6c14d0e10cf190de82,0,0,0,0,0,0,0,1,0,0
2,0x0000000000015b23c7e20b0ea5ebd84c39dcbe60,0,9,0,0,0,0,0,5,0,0
3,0x00000000000360176d958e11c140308cd0863679,9,3,0,0,0,0,8,0,12,0
4,0x000000000004d7463d0f9c77383600bc82d612f5,3,7,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
617327,0xffffe0f5e89ccedadb322fe4ca6bd3ea5badaff2,3,7,0,0,0,0,0,0,0,0
617328,0xffffe59e4ebefce216470864fd92407023288cb4,0,2,0,0,0,0,0,0,0,0
617329,0xfffff6e70842330948ca47254f2be673b1cb0db7,4,0,0,0,0,0,0,7,0,0
617330,0xfffffc03e9d2c7f163135d2b0f18fc7f27c43cce,0,5,0,0,0,0,0,1,0,0


To see that everything checks out let's calculate the sum of every value of the pivot table and compare it against the length of our initial dataframe

In [4]:
pivot_table[pivot_table.columns.drop('buyer_address')].sum().sum()

5252254

In [5]:
len(df)

5252254

Everything works as intended. Let's export our data.

In [6]:
pivot_table.to_csv("nft_association_input.csv",index= False)

The analysis was conducted on RapidMiner, using the following process.

![process](resources/process.png)

To obtain more meaningful results, categories `Uncategorized` and `Collectibles` where excluded. `Uncategorized`, doesn't provide any value and `Collectibles` was considered to be too vague. If deemed necessary of course this can be subject to change.

The association rules found, with thresholds for support equal to 2% and for confidence equal to 10% can be seen in the following picture.

![rules](resources/association_rules_new.png)